Resources:
1. https://github.com/AileenNielsen/PyGotham_2016_Probabilistic_Graphical_Models/blob/master/pgmpy.ipynb
2. https://www.youtube.com/watch?v=DEHqIxX1Kq4&t=1104s 


In [2]:
#pip install pgmpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\programdata\anaconda3\lib\site-packages (from pgmpy) (1.21.5)



In [7]:
#import pgmpy
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork

##### Setting up the structure for the network

In [13]:
model_struct=BayesianNetwork([('Genes','OT'),('Practice','OT'),('OT','Offer')])
model_struct.active_trail_nodes('Genes')

{'Genes': {'Genes', 'OT', 'Offer'}}

#### Relationships or Joint Distribution Probabilities

In [35]:
genes_cpd=TabularCPD(variable='Genes', variable_card = 2, values = [[.2],[.8 ]])
practice_cpd=TabularCPD(variable='Practice', variable_card = 2, values = [[.7],[.3 ]])
offer_cpd=TabularCPD(variable='Offer', variable_card = 2, values = [[.95,0.8,0.5],[.05,0.2,0.5]],evidence_card=[3],
                     evidence=['OT'])
OT_cpd=TabularCPD(variable='OT', variable_card = 3, 
                    values = [[.5, .8, .8, .9],
                            [.3, .15, .1, .08],
                            [.2, .05, .1, .02]],
                    evidence_card=[2,2],
                    evidence=['Genes','Practice'])

In [36]:
model_struct.add_cpds(genes_cpd,practice_cpd,offer_cpd,OT_cpd)

In [38]:
model_struct.get_cpds()

[<TabularCPD representing P(Genes:2) at 0x1a529ca4340>,
 <TabularCPD representing P(Practice:2) at 0x1a529ca4850>,
 <TabularCPD representing P(Offer:2 | OT:3) at 0x1a529ca47f0>,
 <TabularCPD representing P(OT:3 | Genes:2, Practice:2) at 0x1a529ca4c40>]

In [42]:
model_struct.get_independencies()

(Offer ⟂ Practice, Genes | OT)
(Offer ⟂ Genes | OT, Practice)
(Offer ⟂ Practice | OT, Genes)
(Practice ⟂ Genes)
(Practice ⟂ Offer | OT)
(Practice ⟂ Offer | OT, Genes)
(Genes ⟂ Practice)
(Genes ⟂ Offer | OT)
(Genes ⟂ Offer | OT, Practice)

#### Model Inferences

In [52]:
from pgmpy.inference import VariableElimination
model_infer=VariableElimination(model_struct)

# What is the average probability of rolling out an offer
prob_offer=model_infer.query(variables=['Offer'])
print(prob_offer)

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8898 |
+----------+--------------+
| Offer(1) |       0.1102 |
+----------+--------------+


In [59]:
print(model_infer.query(variables=['Offer','Genes']))
print(model_infer.query(variables=['Offer','Practice']))

+----------+----------+--------------------+
| Offer    | Genes    |   phi(Offer,Genes) |
+==========+==========+====================+
| Offer(0) | Genes(0) |             0.1684 |
+----------+----------+--------------------+
| Offer(0) | Genes(1) |             0.7214 |
+----------+----------+--------------------+
| Offer(1) | Genes(0) |             0.0316 |
+----------+----------+--------------------+
| Offer(1) | Genes(1) |             0.0786 |
+----------+----------+--------------------+
+----------+-------------+-----------------------+
| Offer    | Practice    |   phi(Offer,Practice) |
+==========+=============+=======================+
| Offer(0) | Practice(0) |                0.6125 |
+----------+-------------+-----------------------+
| Offer(0) | Practice(1) |                0.2773 |
+----------+-------------+-----------------------+
| Offer(1) | Practice(0) |                0.0875 |
+----------+-------------+-----------------------+
| Offer(1) | Practice(1) |                0.02

In [64]:
print(model_infer.query(variables=['Offer'],evidence={'Genes':0,'Practice':1},show_progress=True))

+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.9050 |
+----------+--------------+
| Offer(1) |       0.0950 |
+----------+--------------+


#### Probable State

In [66]:
model_infer.map_query(['OT'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'OT': 0}